In [13]:
# need 100GB session to run this code (or maybe way less; like just 10GB; idk)
# Using this to make EU-ASPECT files for Yeager. For E3SM SMYLE. Does SE->FV regridding
# Can be used for variety of variables, dims, forecast lengths, etc. 
%reset -f

import xarray as xr
import numpy as np
import glob
import os
from IPython.display import clear_output
from geocat.comp import interp_hybrid_to_pressure

month = "11"
varname = "TREFHT"
varname_out = "tas"
year_start = 1981
year_end = 2021
srcDir0 = "/glade/campaign/cgd/ccr/E3SMv2.1-SMYLE/archive"
srcDir = "/glade/campaign/cgd/ccr/E3SMv2.1-SMYLEsmbb/archive"
destDir = "/glade/derecho/scratch/sglanvil/holdingCell/SMYLE_E3SM"
tmpDir = "/glade/derecho/scratch/sglanvil/holdingCell"
weightFile = "/glade/work/strandwg/E3SM_regridding/maps/map_ne30pg2_to_0.9x1.25_aave.20230801.nc"

data = []
times=[]
for year in range(year_start, year_end + 1):
    clear_output(wait=True)  # Clears the current output
    print(year)
    pattern = f"{srcDir}/M{month}/v21.LR.BSMYLEsmbb.{year}-{month}.*/atm/proc/tseries/month_1/*.eam.h0.{varname}.*.nc"
    if int(month) == 11 and year <= 1989:
        pattern = f"{srcDir0}/v21.LR.BSMYLE.{year}-{month}.*/atm/proc/tseries/month_1/*.eam.h0.{varname}.*.nc"
    memb_files = sorted(glob.glob(pattern))
    memb_data = []
    for inFile in memb_files:
        mappedFile = f"{tmpDir}/tmp_{os.path.basename(inFile)}"
        cmd = f'module load nco && ncremap -m {weightFile} -in {inFile} {mappedFile}'  # ----- Regrid E3SM
        os.system(f'bash -lc "{cmd} > /dev/null 2>&1"')
        ds = xr.open_dataset(mappedFile,decode_times=False)
        os.remove(mappedFile)
        da = ds[varname].expand_dims(M=[0])
        memb_data.append(da)
    year_data = xr.concat(memb_data, dim="M").expand_dims(Y=[year])
    data.append(year_data)
    times.append(xr.open_dataset(memb_files[0])["time"].drop_vars("time", errors="ignore").expand_dims("Y"))
data = xr.concat(data, dim = "Y").rename({"time": "L"})
times = xr.concat(times, dim="Y").rename({"time": "L"})
ds = data.to_dataset(name=varname_out)
ds["time"] = times
ds = ds.assign_coords(M=np.arange(1, ds.sizes["M"] + 1))
ds = ds.assign_coords(L=np.arange(1, ds.sizes["L"] + 1))
ds.to_netcdf(f"{destDir}/{varname_out}_{year_start}-{year_end}_M{month}_E3SM_SMYLE.nc")


2021


### Deal with total precip calculation (PRECC+PRECL)

In [14]:
# Deal with total precip calculation (PRECC+PRECL)
month = "11"
ds_precc = xr.open_dataset(f"{destDir}/PRECC_1981-2021_M{month}_E3SM_SMYLE.nc")
ds_precl = xr.open_dataset(f"{destDir}/PRECL_1981-2021_M{month}_E3SM_SMYLE.nc")
pr = ds_precc["PRECC"] + ds_precl["PRECL"]
pr.attrs = ds_precc["PRECC"].attrs
pr.attrs["long_name"] = "Total precipitation (convective + large-scale)"
ds = pr.to_dataset(name="pr")
ds.to_netcdf(f"{destDir}/pr_{year_start}-{year_end}_M{month}_E3SM_SMYLE.nc")


### Concatenate already-mapped, already-single-level U and V (made from bash + .py scripts)

In [26]:
# first: /glade/derecho/scratch/sglanvil/holdingCell/E3SM_mapped
# 1. loop_map_files.sh  -> map_files.sh
# 2. loop0_hybrid_to_pressure.sh  -> loop_hybrid_to_pressure.sh  -> hybrid_to_pressure.py
%reset -f

import xarray as xr
import numpy as np
import glob
import os
from IPython.display import clear_output
from geocat.comp import interp_hybrid_to_pressure

month = "11"
varname = "V250"
varname_out = "va250"
year_start = 1981
year_end = 2021
srcDir = f"/glade/derecho/scratch/sglanvil/holdingCell/E3SM_mapped/{varname}"
destDir = "/glade/derecho/scratch/sglanvil/holdingCell/SMYLE_E3SM"

data = []
times=[]
for year in range(year_start, year_end + 1):
    clear_output(wait=True)  # Clears the current output
    print(year)
    pattern = f"{srcDir}/mapped_v21.LR.BSMYLEsmbb.{year}-{month}.*.eam.h0.{varname}.*.nc"
    memb_files = sorted(glob.glob(pattern))
    memb_data = []
    for inFile in memb_files:
        ds = xr.open_dataset(inFile,decode_times=False)
        da = ds[varname].expand_dims(M=[0])
        memb_data.append(da)
    year_data = xr.concat(memb_data, dim="M").expand_dims(Y=[year])
    data.append(year_data)
    times.append(xr.open_dataset(memb_files[0])["time"].drop_vars("time", errors="ignore").expand_dims("Y"))
data = xr.concat(data, dim = "Y").rename({"time": "L"})
times = xr.concat(times, dim="Y").rename({"time": "L"})
ds = data.to_dataset(name=varname_out)
ds["time"] = times
ds = ds.assign_coords(M=np.arange(1, ds.sizes["M"] + 1))
ds = ds.assign_coords(L=np.arange(1, ds.sizes["L"] + 1))
ds.to_netcdf(f"{destDir}/{varname_out}_{year_start}-{year_end}_M{month}_E3SM_SMYLE.nc")

# final location for EU-ASPECT: /glade/campaign/cgd/ccr/E3SMv2.1-SMYLEsmbb/EU-ASPECT_submission
# yeager stuff: /glade/u/home/yeager/analysis/python/SMYLE-analysis/notebooks/SMYLE_for_EU-ASPECT
# yeager stuff: /glade/u/home/yeager/analysis/python/CATALYST/E3SM-SMYLE/SeasonalSkill/SMYLE.seasSAT.computeskill_tocompare.ipynb

2021


### Calculate and concatenate divergence at 200mb (for CESM, TBI-PACEMAKER, not E3SM)

In [68]:
import xarray as xr
import numpy as np
import glob
import os
from IPython.display import clear_output

# 1. CTRL
# 2. TBI-ATL
# 3. TBI-PAC
# 4. TBI-IND
# dimensions:  monthly data, Y=1982-2021, L=23, M=10 (Except control M=20)
# TBI: /glade/campaign/cesm/development/espwg/SMYLE-PACEMAKER/archive/
# CTRL: /glade/campaign/cesm/development/espwg/SMYLE/archive/b.e21.BSMYLE.f09_g17.1984-02.015/atm/proc/tseries/month_1

month = "02"
year_start = 1982
year_end = 2021
srcDir = "/glade/derecho/scratch/sglanvil/holdingCell/CESM_mapped"
destDir = "/glade/derecho/scratch/sglanvil/holdingCell/CESM_mapped/div200"
os.makedirs(destDir, exist_ok=True)

data = []
times=[]
for year in range(year_start, year_end + 1):
    print(year)
    pattern = f"{srcDir}/U200/b.e21.BSMYLE.f09_g17.{year}-{month}.*.cam.h0.U200.*.nc"
    memb_files = sorted(glob.glob(pattern))
    memb_data = []
    for U200file in memb_files[:20]: # just grab the first 20 members (or less)
        V200file = U200file.replace("U200", "V200")
        ds_u = xr.open_dataset(U200file,decode_times=False)
        ds_v = xr.open_dataset(V200file,decode_times=False)
        U200 = ds_u['U200']
        V200 = ds_v['V200']
        lat_rad = np.deg2rad(U200.lat.values)  
        lon_rad = np.deg2rad(U200.lon.values)
        lon2d, lat2d = np.meshgrid(lon_rad, lat_rad)
        dlat = np.gradient(lat_rad)  
        dlon = np.gradient(lon_rad)
        R = 6.371e6
        dx = R * np.cos(lat2d) * dlon[np.newaxis, :]
        dy = R * dlat[:, np.newaxis]
        # dUdx = np.gradient(U200, axis=2) / dx
        # dVdy = np.gradient(V200, axis=1) / dy 
        # div200 = dUdx + dVdy 
        v_coslat = V200.values * np.cos(lat2d)
        dUdlon = np.gradient(U200.values, axis=2) / dx
        dvcos_dlat = np.gradient(v_coslat, axis=1) / dy
        div200 = dUdlon + dvcos_dlat
        div200_xr = xr.DataArray(div200, dims=U200.dims, coords=U200.coords, name="div200")
        div200_xr.attrs = {
            "units": "1/s",
            "long_name": "Horizontal divergence at 200 mb",
            "description": "Computed from U200 and V200 using spherical coordinates",}
        da = div200_xr.expand_dims(M=[0])
        memb_data.append(da)
    year_data = xr.concat(memb_data, dim="M").expand_dims(Y=[year])
    data.append(year_data)
    times.append(xr.open_dataset(memb_files[0])["time"].drop_vars("time", errors="ignore").expand_dims("Y"))
data = xr.concat(data, dim = "Y").rename({"time": "L"})
times = xr.concat(times, dim="Y").rename({"time": "L"})
ds = data.to_dataset(name="div200")
ds["time"] = times
ds = ds.assign_coords(M=np.arange(1, ds.sizes["M"] + 1))
ds = ds.assign_coords(L=np.arange(1, ds.sizes["L"] + 1))
ds.to_netcdf(f"{destDir}/smyle_ctrl_div200.nc")


1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021


### Concatenate U at 10m (for CESM, TBI-PACEMAKER, not E3SM)

In [9]:
%reset -f

import xarray as xr
import numpy as np
import glob
import os
from IPython.display import clear_output
from geocat.comp import interp_hybrid_to_pressure

month = "02"
varname = "U990"
varname_out = "U10m"
year_start = 1982
year_end = 2021

srcDir = f"/glade/derecho/scratch/sglanvil/holdingCell/CESM_mapped/{varname}"
destDir = f"/glade/derecho/scratch/sglanvil/holdingCell/CESM_mapped/{varname_out}"
os.makedirs(destDir, exist_ok=True)

data = []
times=[]
for year in range(year_start, year_end + 1):
    clear_output(wait=True)  # Clears the current output
    print(year)
    # pattern = f"{srcDir}/b.e21.BSMYLE.f09_g17.{year}-{month}.*.cam.h0.{varname}.*.nc"
    pattern = f"{srcDir}/b.e21.BSMYLE.f09_g17.TBI-IND_15d50m-anom.{year}-{month}.*.cam.h0.{varname}.*.nc"
    memb_files = sorted(glob.glob(pattern))
    memb_data = []
    for inFile in memb_files[:20]:
        ds = xr.open_dataset(inFile,decode_times=False)
        da = ds[varname].expand_dims(M=[0])
        memb_data.append(da)
    year_data = xr.concat(memb_data, dim="M").expand_dims(Y=[year])
    data.append(year_data)
    times.append(xr.open_dataset(memb_files[0])["time"].drop_vars("time", errors="ignore").expand_dims("Y"))
data = xr.concat(data, dim = "Y").rename({"time": "L"})
times = xr.concat(times, dim="Y").rename({"time": "L"})
ds = data.to_dataset(name=varname_out)
ds["time"] = times
ds = ds.assign_coords(M=np.arange(1, ds.sizes["M"] + 1))
ds = ds.assign_coords(L=np.arange(1, ds.sizes["L"] + 1))
ds.to_netcdf(f"{destDir}/tbi_ind_{varname_out}.nc")


2021
